# **Automatic text summarizer_sumy**
- sumy（正式名：Automatic text summarizer）は抽出型の要約を行うことができるPythonのライブラリ。
- sumyは、HTMLページやプレーンテキストから要約を抽出するためのシンプルなライブラリとコマンドラインユーティリティです。


---
- **長文を、意図した長さ（センテンス）に、複数のアルゴリズムで要約**することができます。
- <font color= "LightCoral">**ライブラリのインストールとインポートは数分かかります。**</font>気長に待ちましょう。
- **「ランタイム→すべてのセルを実行」**し、ライブラリが読み込まれた後、**[ファイル選択] ボタンをクリックし、要約したいテキストファイル（.txt）指定するだけ**です。
- **要約結果**は、アルゴリズム毎に「原文書」「要約文書」が表示されます。**全表示**では、原文書とすべてのアルゴリズム（LexRank, Lsa, Reduction, Luhn, SumBasic, KL）の要約結果が確認できます。
- 要約センテンス数：要約したいセンテンス数をスライドバーにて設定できます。※設定幅：3～30（Default=3, 1Step）
- <font color= "LightCoral">**[注意]**</font> テキストファイルは文字コードを「UTF-8」としてください。
- <font color= "LightCoral">**[注意]** 「ライブラリのインストール＆インポート」でエラーとなる場合がありました。</font> GINZAをアンインストールし、再インストール（ランタイム再起動）すると復帰しました。最初のセルの#を外すとこれが実行できます。
- <font color= "LightCoral">**[注意]** テキストの内容によって特定のアルゴリズムのみ要約エラーとなる場合がありました。</font> エラー箇所が表示されますので、必要に応じ該当箇所を修正したり、変更したり、カットしたりしてください。

# **ライブラリインストール & インポート**

In [1]:
pip uninstall ginza ja-ginza

In [2]:
!pip install sudachipy sudachidict_core
!pip install -U ginza https://github.com/megagonlabs/ginza/releases/download/latest/ja_ginza_electra-latest-with-model.tar.gz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.2 MB 5.1 MB/s 
  Created wheel for sudachidict-core: filename=SudachiDict_core-20220519-py3-none-any.whl size=71522141 sha256=9e98452467f62b75287c37197017a1c0c36cbe1dad337133a8677c0bf8fb8a8a
  Stored in directory: /root/.cache/pip/wheels/64/d8/db/4e4d15ae9c54b1956670125c5ece6fb6b93e4fc081f36450f9
Successfully built sudachidict-core
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 809.7 MB 4.6 kB/s 
     |████████████████████████████████| 6.0 MB 4.8 MB/s 
     |████████████████████████████████| 283 kB 45.9 MB/s 
     |████████████████████████████████| 51 kB 137 kB/s 
     |████████████████████████████████| 3.1 MB 30.5 MB/s 
     |████████████████████████████████| 42 kB 1.2 MB/s 
     |████████████████████████████████| 181 kB 44.1 MB/s 
     |██████████████████

In [3]:
import spacy
nlp = spacy.load('ja_ginza_electra')

In [4]:
pip install sumy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 94 kB 1.8 MB/s 
     |████████████████████████████████| 10.1 MB 31.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for breadability: filename=breadability-0.1.20-py2.py3-none-any.whl size=21712 sha256=452badc0ddde985316b17209f963fa421a45cf9c1ae843127601f935e7081f02
  Stored in directory: /root/.cache/pip/wheels/d4/bf/51/81d27ad638e1a6dca4f362ecc33d1e2c764b8ea7ec751b8fc1
  Created wheel for pycountry: filename=pycountry-22.3.5-py2.py3-none-any.whl size=10681845 sha256=337144b9e60edb4fcab759ed8d3edbdb10592c77f7257612c90c5c194e1706e3
  Stored in directory: /root/.cache/pip/wheels/0e/06/e8/7ee176e95ea9a8a8c3b3afcb1869f20adbd42413d4611c6eb4
Successfully built breadability pycountry


In [5]:
pip install tinysegmenter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for tinysegmenter: filename=tinysegmenter-0.4-py3-none-any.whl size=13552 sha256=de439d7268ea3f50c5768c81a9ea9c26e08cd5d92afeba67d31c644227c3fbf8
  Stored in directory: /root/.cache/pip/wheels/bb/a1/ac/d3d490f093f522c2f9e116ec2604480f2c9d02d85b96773ee5
Successfully built tinysegmenter


In [6]:
import re

# **テキスト読込み**

In [7]:
from google.colab import files
print('txtファイル(UTF-8）を指定してください')
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))

txtファイル(UTF-8）を指定してください


Saving heavy.txt to heavy.txt
User uploaded file "heavy.txt" with length 32057 bytes


In [8]:
#@title 要約センテンス数設定（Defalt = 3） { run: "auto" }
Sentences = 3 #@param {type:"slider", min:3, max:30, step:1}

# **要約処理**

In [9]:
if len(uploaded.keys()) != 1:
    print("アップロードは１ファイルにのみ限ります")
else:
    target = list(uploaded.keys())[0]
    
with open(target) as f:
    contents = f.readlines()

document = ''.join(contents).replace(' ', '').replace('　', '')

In [10]:
corpus = []
originals = []
doc = nlp(document)
for s in doc.sents:
    originals.append(s)
    tokens = []
    for t in s:
        tokens.append(t.lemma_)
    corpus.append(' '.join(tokens))

#print(len(corpus))
#print(len(originals))

/usr/local/lib/python3.7/dist-packages/torch/autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


In [11]:
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lex_rank import LexRankSummarizer
from sumy.summarizers.lsa import LsaSummarizer
from sumy.summarizers.reduction import ReductionSummarizer
from sumy.summarizers.luhn import LuhnSummarizer
from sumy.summarizers.sum_basic import SumBasicSummarizer
from sumy.summarizers.kl import KLSummarizer

# 連結したcorpusを再度tinysegmenterでトークナイズさせる
parser = PlaintextParser.from_string(''.join(corpus), Tokenizer('japanese'))

summarizer = LexRankSummarizer()
summarizer.stop_words = [' ']  # スペースも1単語として認識されるため、ストップワードにすることで除外する

# sentencres_count指定
summary = summarizer(document=parser.document, sentences_count=Sentences)

# 原文書を表示
print(u'[原文書]')
print(document)

# 要約文を表示
print(u'[要約文書]')
for sentence in summary:
    print(originals[corpus.index(sentence.__str__())])

[原文書]
ハードロック/ヘヴィメタルは70年代半ばごろから、アリーナ・ロックや産業ロック的なバンドと、アルバム志向のヘヴィメタル・バンドに分かれる傾向も見られ、また、時代が新しくなるにしたがって、シーンも細分化が進んできた。シーンの細分化によって、ヘヴィメタルは様々な「サブジャンル」を持つようになった。このジャンルに分類されるバンドのサウンドはハードロック[5]同様、エレクトリック・ギターのファズやディストーションを強調した、ラウドなものであるのが基本である。ヘヴィメタルという用語自体は、70年前半から存在したが、ハードロックが70年代前半にピークを迎えた後、パンク・ロックの性急感を加えて存続したジャンルである。ハードロックとヘヴィメタルの間に厳密な境界線は無く、ハードロックとヘヴィメタルとを一括りにしてHR/HM（HM/HR）と呼ぶこともある。メンバー構成は、ロックバンド一般に見られるものとあまり変わらないことが多い。ギター、ドラム、ボーカル、ベースを主軸にし、これにキーボードが加わることもある。ジャンル名の通り、一般的には音の「ヘヴィさ」が重視される。そのためにギターやベースのチューニングを下げて、通常より低い音が出せるようにしている場合がある。ヘヴィメタルではギターソロが重視される場合が多く、たいていの場合は一曲の間にギターソロが挿入される。また、ドラムソロやベースソロも行われることも多く、歌よりも演奏で魅せるような曲も多く、インストゥルメンタルの曲も多い。こういった傾向から、速弾きなどのテクニカルな演奏を得意とするプレイヤーを多く生み出しており、エフェクターなど音楽機材の進化と多様化に多大な影響を与えたとも言われている。代表的なギタリストには、ジミー・ペイジ[6]、トニー・アイオミ、エドワード・ヴァン・ヘイレン、マイケル・シェンカー、アンガス・ヤング、イングヴェイ・マルムスティーン、スティーヴ・ヴァイらがいる。通例、強いディストーションをかけ、リフはパワーコードを主体とした力強い音でミュートを効かせながら刻む場合が多い。ヘヴィメタルバンドにはギタリストが2人いることが多い。リズムギター担当とリードギター担当に分かれている場合と、2人が同じリフを弾いて重さを増している場合や、2人が交互にギターソロを弾くこともある。スケールにはペンタトニック、ハーモニック・マ

### **GiNZA固有表現抽出**

In [12]:
#GiNZA固有表現抽出
doc = nlp(document)
spacy.displacy.render(doc, style="ent", jupyter=True)

/usr/local/lib/python3.7/dist-packages/torch/autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


# **要約結果**

### **LexRank**
- 抽出型に分類される要約アルゴリズム、文書からグラフ構造を作り出して重要な文のランキングを作ることで要約と言える文を発見する

In [13]:
print(u'[原文書]')
print(document)
print(u'[要約文書:LexRank]')
for sentence in summary:
    print(originals[corpus.index(sentence.__str__())])

[原文書]
ハードロック/ヘヴィメタルは70年代半ばごろから、アリーナ・ロックや産業ロック的なバンドと、アルバム志向のヘヴィメタル・バンドに分かれる傾向も見られ、また、時代が新しくなるにしたがって、シーンも細分化が進んできた。シーンの細分化によって、ヘヴィメタルは様々な「サブジャンル」を持つようになった。このジャンルに分類されるバンドのサウンドはハードロック[5]同様、エレクトリック・ギターのファズやディストーションを強調した、ラウドなものであるのが基本である。ヘヴィメタルという用語自体は、70年前半から存在したが、ハードロックが70年代前半にピークを迎えた後、パンク・ロックの性急感を加えて存続したジャンルである。ハードロックとヘヴィメタルの間に厳密な境界線は無く、ハードロックとヘヴィメタルとを一括りにしてHR/HM（HM/HR）と呼ぶこともある。メンバー構成は、ロックバンド一般に見られるものとあまり変わらないことが多い。ギター、ドラム、ボーカル、ベースを主軸にし、これにキーボードが加わることもある。ジャンル名の通り、一般的には音の「ヘヴィさ」が重視される。そのためにギターやベースのチューニングを下げて、通常より低い音が出せるようにしている場合がある。ヘヴィメタルではギターソロが重視される場合が多く、たいていの場合は一曲の間にギターソロが挿入される。また、ドラムソロやベースソロも行われることも多く、歌よりも演奏で魅せるような曲も多く、インストゥルメンタルの曲も多い。こういった傾向から、速弾きなどのテクニカルな演奏を得意とするプレイヤーを多く生み出しており、エフェクターなど音楽機材の進化と多様化に多大な影響を与えたとも言われている。代表的なギタリストには、ジミー・ペイジ[6]、トニー・アイオミ、エドワード・ヴァン・ヘイレン、マイケル・シェンカー、アンガス・ヤング、イングヴェイ・マルムスティーン、スティーヴ・ヴァイらがいる。通例、強いディストーションをかけ、リフはパワーコードを主体とした力強い音でミュートを効かせながら刻む場合が多い。ヘヴィメタルバンドにはギタリストが2人いることが多い。リズムギター担当とリードギター担当に分かれている場合と、2人が同じリフを弾いて重さを増している場合や、2人が交互にギターソロを弾くこともある。スケールにはペンタトニック、ハーモニック・マ

### **Lsa**
- Latent Semantic Analysis：ベクトル空間モデルを利用した自然言語処理の技法の1つで、文書群とそこに含まれる用語群について、それらに関連した概念の集合を生成することで、その関係を分析する技術

In [14]:
summarizer = LsaSummarizer()
summarizer.stop_words = [' ']  # スペースも1単語として認識されるため、ストップワードにすることで除外する
# sentencres_count指定
summary = summarizer(document=parser.document, sentences_count=Sentences)

print(u'[原文書]')
print(document)
print(u'[要約文書:Lsa]')
for sentence in summary:
    print(originals[corpus.index(sentence.__str__())])

[原文書]
ハードロック/ヘヴィメタルは70年代半ばごろから、アリーナ・ロックや産業ロック的なバンドと、アルバム志向のヘヴィメタル・バンドに分かれる傾向も見られ、また、時代が新しくなるにしたがって、シーンも細分化が進んできた。シーンの細分化によって、ヘヴィメタルは様々な「サブジャンル」を持つようになった。このジャンルに分類されるバンドのサウンドはハードロック[5]同様、エレクトリック・ギターのファズやディストーションを強調した、ラウドなものであるのが基本である。ヘヴィメタルという用語自体は、70年前半から存在したが、ハードロックが70年代前半にピークを迎えた後、パンク・ロックの性急感を加えて存続したジャンルである。ハードロックとヘヴィメタルの間に厳密な境界線は無く、ハードロックとヘヴィメタルとを一括りにしてHR/HM（HM/HR）と呼ぶこともある。メンバー構成は、ロックバンド一般に見られるものとあまり変わらないことが多い。ギター、ドラム、ボーカル、ベースを主軸にし、これにキーボードが加わることもある。ジャンル名の通り、一般的には音の「ヘヴィさ」が重視される。そのためにギターやベースのチューニングを下げて、通常より低い音が出せるようにしている場合がある。ヘヴィメタルではギターソロが重視される場合が多く、たいていの場合は一曲の間にギターソロが挿入される。また、ドラムソロやベースソロも行われることも多く、歌よりも演奏で魅せるような曲も多く、インストゥルメンタルの曲も多い。こういった傾向から、速弾きなどのテクニカルな演奏を得意とするプレイヤーを多く生み出しており、エフェクターなど音楽機材の進化と多様化に多大な影響を与えたとも言われている。代表的なギタリストには、ジミー・ペイジ[6]、トニー・アイオミ、エドワード・ヴァン・ヘイレン、マイケル・シェンカー、アンガス・ヤング、イングヴェイ・マルムスティーン、スティーヴ・ヴァイらがいる。通例、強いディストーションをかけ、リフはパワーコードを主体とした力強い音でミュートを効かせながら刻む場合が多い。ヘヴィメタルバンドにはギタリストが2人いることが多い。リズムギター担当とリードギター担当に分かれている場合と、2人が同じリフを弾いて重さを増している場合や、2人が交互にギターソロを弾くこともある。スケールにはペンタトニック、ハーモニック・マ

### **Reduction**

In [15]:
summarizer = ReductionSummarizer()
summarizer.stop_words = [' ']  # スペースも1単語として認識されるため、ストップワードにすることで除外する
# sentencres_count指定
summary = summarizer(document=parser.document, sentences_count=Sentences)

print(u'[原文書]')
print(document)
print(u'[要約文書:Reduction]')
for sentence in summary:
    print(originals[corpus.index(sentence.__str__())])

[原文書]
ハードロック/ヘヴィメタルは70年代半ばごろから、アリーナ・ロックや産業ロック的なバンドと、アルバム志向のヘヴィメタル・バンドに分かれる傾向も見られ、また、時代が新しくなるにしたがって、シーンも細分化が進んできた。シーンの細分化によって、ヘヴィメタルは様々な「サブジャンル」を持つようになった。このジャンルに分類されるバンドのサウンドはハードロック[5]同様、エレクトリック・ギターのファズやディストーションを強調した、ラウドなものであるのが基本である。ヘヴィメタルという用語自体は、70年前半から存在したが、ハードロックが70年代前半にピークを迎えた後、パンク・ロックの性急感を加えて存続したジャンルである。ハードロックとヘヴィメタルの間に厳密な境界線は無く、ハードロックとヘヴィメタルとを一括りにしてHR/HM（HM/HR）と呼ぶこともある。メンバー構成は、ロックバンド一般に見られるものとあまり変わらないことが多い。ギター、ドラム、ボーカル、ベースを主軸にし、これにキーボードが加わることもある。ジャンル名の通り、一般的には音の「ヘヴィさ」が重視される。そのためにギターやベースのチューニングを下げて、通常より低い音が出せるようにしている場合がある。ヘヴィメタルではギターソロが重視される場合が多く、たいていの場合は一曲の間にギターソロが挿入される。また、ドラムソロやベースソロも行われることも多く、歌よりも演奏で魅せるような曲も多く、インストゥルメンタルの曲も多い。こういった傾向から、速弾きなどのテクニカルな演奏を得意とするプレイヤーを多く生み出しており、エフェクターなど音楽機材の進化と多様化に多大な影響を与えたとも言われている。代表的なギタリストには、ジミー・ペイジ[6]、トニー・アイオミ、エドワード・ヴァン・ヘイレン、マイケル・シェンカー、アンガス・ヤング、イングヴェイ・マルムスティーン、スティーヴ・ヴァイらがいる。通例、強いディストーションをかけ、リフはパワーコードを主体とした力強い音でミュートを効かせながら刻む場合が多い。ヘヴィメタルバンドにはギタリストが2人いることが多い。リズムギター担当とリードギター担当に分かれている場合と、2人が同じリフを弾いて重さを増している場合や、2人が交互にギターソロを弾くこともある。スケールにはペンタトニック、ハーモニック・マ

### **Luhn**

In [16]:
summarizer = LuhnSummarizer()
summarizer.stop_words = [' ']  # スペースも1単語として認識されるため、ストップワードにすることで除外する
# sentencres_count指定
summary = summarizer(document=parser.document, sentences_count=Sentences)

print(u'[原文書]')
print(document)
print(u'[要約文書:Luhn]')
for sentence in summary:
    print(originals[corpus.index(sentence.__str__())])

[原文書]
ハードロック/ヘヴィメタルは70年代半ばごろから、アリーナ・ロックや産業ロック的なバンドと、アルバム志向のヘヴィメタル・バンドに分かれる傾向も見られ、また、時代が新しくなるにしたがって、シーンも細分化が進んできた。シーンの細分化によって、ヘヴィメタルは様々な「サブジャンル」を持つようになった。このジャンルに分類されるバンドのサウンドはハードロック[5]同様、エレクトリック・ギターのファズやディストーションを強調した、ラウドなものであるのが基本である。ヘヴィメタルという用語自体は、70年前半から存在したが、ハードロックが70年代前半にピークを迎えた後、パンク・ロックの性急感を加えて存続したジャンルである。ハードロックとヘヴィメタルの間に厳密な境界線は無く、ハードロックとヘヴィメタルとを一括りにしてHR/HM（HM/HR）と呼ぶこともある。メンバー構成は、ロックバンド一般に見られるものとあまり変わらないことが多い。ギター、ドラム、ボーカル、ベースを主軸にし、これにキーボードが加わることもある。ジャンル名の通り、一般的には音の「ヘヴィさ」が重視される。そのためにギターやベースのチューニングを下げて、通常より低い音が出せるようにしている場合がある。ヘヴィメタルではギターソロが重視される場合が多く、たいていの場合は一曲の間にギターソロが挿入される。また、ドラムソロやベースソロも行われることも多く、歌よりも演奏で魅せるような曲も多く、インストゥルメンタルの曲も多い。こういった傾向から、速弾きなどのテクニカルな演奏を得意とするプレイヤーを多く生み出しており、エフェクターなど音楽機材の進化と多様化に多大な影響を与えたとも言われている。代表的なギタリストには、ジミー・ペイジ[6]、トニー・アイオミ、エドワード・ヴァン・ヘイレン、マイケル・シェンカー、アンガス・ヤング、イングヴェイ・マルムスティーン、スティーヴ・ヴァイらがいる。通例、強いディストーションをかけ、リフはパワーコードを主体とした力強い音でミュートを効かせながら刻む場合が多い。ヘヴィメタルバンドにはギタリストが2人いることが多い。リズムギター担当とリードギター担当に分かれている場合と、2人が同じリフを弾いて重さを増している場合や、2人が交互にギターソロを弾くこともある。スケールにはペンタトニック、ハーモニック・マ

### **SumBasic**

In [17]:
summarizer = SumBasicSummarizer()
summarizer.stop_words = [' ']  # スペースも1単語として認識されるため、ストップワードにすることで除外する
# sentencres_count指定
summary = summarizer(document=parser.document, sentences_count=Sentences)

print(u'[原文書]')
print(document)
print(u'[要約文書:SumBasic]')
for sentence in summary:
    print(originals[corpus.index(sentence.__str__())])

[原文書]
ハードロック/ヘヴィメタルは70年代半ばごろから、アリーナ・ロックや産業ロック的なバンドと、アルバム志向のヘヴィメタル・バンドに分かれる傾向も見られ、また、時代が新しくなるにしたがって、シーンも細分化が進んできた。シーンの細分化によって、ヘヴィメタルは様々な「サブジャンル」を持つようになった。このジャンルに分類されるバンドのサウンドはハードロック[5]同様、エレクトリック・ギターのファズやディストーションを強調した、ラウドなものであるのが基本である。ヘヴィメタルという用語自体は、70年前半から存在したが、ハードロックが70年代前半にピークを迎えた後、パンク・ロックの性急感を加えて存続したジャンルである。ハードロックとヘヴィメタルの間に厳密な境界線は無く、ハードロックとヘヴィメタルとを一括りにしてHR/HM（HM/HR）と呼ぶこともある。メンバー構成は、ロックバンド一般に見られるものとあまり変わらないことが多い。ギター、ドラム、ボーカル、ベースを主軸にし、これにキーボードが加わることもある。ジャンル名の通り、一般的には音の「ヘヴィさ」が重視される。そのためにギターやベースのチューニングを下げて、通常より低い音が出せるようにしている場合がある。ヘヴィメタルではギターソロが重視される場合が多く、たいていの場合は一曲の間にギターソロが挿入される。また、ドラムソロやベースソロも行われることも多く、歌よりも演奏で魅せるような曲も多く、インストゥルメンタルの曲も多い。こういった傾向から、速弾きなどのテクニカルな演奏を得意とするプレイヤーを多く生み出しており、エフェクターなど音楽機材の進化と多様化に多大な影響を与えたとも言われている。代表的なギタリストには、ジミー・ペイジ[6]、トニー・アイオミ、エドワード・ヴァン・ヘイレン、マイケル・シェンカー、アンガス・ヤング、イングヴェイ・マルムスティーン、スティーヴ・ヴァイらがいる。通例、強いディストーションをかけ、リフはパワーコードを主体とした力強い音でミュートを効かせながら刻む場合が多い。ヘヴィメタルバンドにはギタリストが2人いることが多い。リズムギター担当とリードギター担当に分かれている場合と、2人が同じリフを弾いて重さを増している場合や、2人が交互にギターソロを弾くこともある。スケールにはペンタトニック、ハーモニック・マ

### **KL**

In [18]:
summarizer = KLSummarizer()
summarizer.stop_words = [' ']  # スペースも1単語として認識されるため、ストップワードにすることで除外する
# sentencres_count指定
summary = summarizer(document=parser.document, sentences_count=Sentences)

print(u'[原文書]')
print(document)
print(u'[要約文書:KL]')
for sentence in summary:
    print(originals[corpus.index(sentence.__str__())])

[原文書]
ハードロック/ヘヴィメタルは70年代半ばごろから、アリーナ・ロックや産業ロック的なバンドと、アルバム志向のヘヴィメタル・バンドに分かれる傾向も見られ、また、時代が新しくなるにしたがって、シーンも細分化が進んできた。シーンの細分化によって、ヘヴィメタルは様々な「サブジャンル」を持つようになった。このジャンルに分類されるバンドのサウンドはハードロック[5]同様、エレクトリック・ギターのファズやディストーションを強調した、ラウドなものであるのが基本である。ヘヴィメタルという用語自体は、70年前半から存在したが、ハードロックが70年代前半にピークを迎えた後、パンク・ロックの性急感を加えて存続したジャンルである。ハードロックとヘヴィメタルの間に厳密な境界線は無く、ハードロックとヘヴィメタルとを一括りにしてHR/HM（HM/HR）と呼ぶこともある。メンバー構成は、ロックバンド一般に見られるものとあまり変わらないことが多い。ギター、ドラム、ボーカル、ベースを主軸にし、これにキーボードが加わることもある。ジャンル名の通り、一般的には音の「ヘヴィさ」が重視される。そのためにギターやベースのチューニングを下げて、通常より低い音が出せるようにしている場合がある。ヘヴィメタルではギターソロが重視される場合が多く、たいていの場合は一曲の間にギターソロが挿入される。また、ドラムソロやベースソロも行われることも多く、歌よりも演奏で魅せるような曲も多く、インストゥルメンタルの曲も多い。こういった傾向から、速弾きなどのテクニカルな演奏を得意とするプレイヤーを多く生み出しており、エフェクターなど音楽機材の進化と多様化に多大な影響を与えたとも言われている。代表的なギタリストには、ジミー・ペイジ[6]、トニー・アイオミ、エドワード・ヴァン・ヘイレン、マイケル・シェンカー、アンガス・ヤング、イングヴェイ・マルムスティーン、スティーヴ・ヴァイらがいる。通例、強いディストーションをかけ、リフはパワーコードを主体とした力強い音でミュートを効かせながら刻む場合が多い。ヘヴィメタルバンドにはギタリストが2人いることが多い。リズムギター担当とリードギター担当に分かれている場合と、2人が同じリフを弾いて重さを増している場合や、2人が交互にギターソロを弾くこともある。スケールにはペンタトニック、ハーモニック・マ

### **全表示**

In [19]:
print(u'[原文書]')
print(document)

print(u'[要約文書:LexRank]')
for sentence in summary:
    print(originals[corpus.index(sentence.__str__())])

summarizer = LsaSummarizer()
summarizer.stop_words = [' ']  # スペースも1単語として認識されるため、ストップワードにすることで除外する
# sentencres_count指定
summary = summarizer(document=parser.document, sentences_count=Sentences)

print(u'[要約文書:Lsa]')
for sentence in summary:
    print(originals[corpus.index(sentence.__str__())])

summarizer = ReductionSummarizer()
summarizer.stop_words = [' ']  # スペースも1単語として認識されるため、ストップワードにすることで除外する
# sentencres_count指定
summary = summarizer(document=parser.document, sentences_count=Sentences)

print(u'[要約文書:Reduction]')
for sentence in summary:
    print(originals[corpus.index(sentence.__str__())])

summarizer = LuhnSummarizer()
summarizer.stop_words = [' ']  # スペースも1単語として認識されるため、ストップワードにすることで除外する
# sentencres_count指定
summary = summarizer(document=parser.document, sentences_count=Sentences)

print(u'[要約文書:Luhn]')
for sentence in summary:
    print(originals[corpus.index(sentence.__str__())])

summarizer = SumBasicSummarizer()
summarizer.stop_words = [' ']  # スペースも1単語として認識されるため、ストップワードにすることで除外する
# sentencres_count指定
summary = summarizer(document=parser.document, sentences_count=Sentences)

print(u'[要約文書:SumBasic]')
for sentence in summary:
    print(originals[corpus.index(sentence.__str__())])

summarizer = KLSummarizer()
summarizer.stop_words = [' ']  # スペースも1単語として認識されるため、ストップワードにすることで除外する
# sentencres_count指定
summary = summarizer(document=parser.document, sentences_count=Sentences)

print(u'[要約文書:KL]')
for sentence in summary:
    print(originals[corpus.index(sentence.__str__())])

[原文書]
ハードロック/ヘヴィメタルは70年代半ばごろから、アリーナ・ロックや産業ロック的なバンドと、アルバム志向のヘヴィメタル・バンドに分かれる傾向も見られ、また、時代が新しくなるにしたがって、シーンも細分化が進んできた。シーンの細分化によって、ヘヴィメタルは様々な「サブジャンル」を持つようになった。このジャンルに分類されるバンドのサウンドはハードロック[5]同様、エレクトリック・ギターのファズやディストーションを強調した、ラウドなものであるのが基本である。ヘヴィメタルという用語自体は、70年前半から存在したが、ハードロックが70年代前半にピークを迎えた後、パンク・ロックの性急感を加えて存続したジャンルである。ハードロックとヘヴィメタルの間に厳密な境界線は無く、ハードロックとヘヴィメタルとを一括りにしてHR/HM（HM/HR）と呼ぶこともある。メンバー構成は、ロックバンド一般に見られるものとあまり変わらないことが多い。ギター、ドラム、ボーカル、ベースを主軸にし、これにキーボードが加わることもある。ジャンル名の通り、一般的には音の「ヘヴィさ」が重視される。そのためにギターやベースのチューニングを下げて、通常より低い音が出せるようにしている場合がある。ヘヴィメタルではギターソロが重視される場合が多く、たいていの場合は一曲の間にギターソロが挿入される。また、ドラムソロやベースソロも行われることも多く、歌よりも演奏で魅せるような曲も多く、インストゥルメンタルの曲も多い。こういった傾向から、速弾きなどのテクニカルな演奏を得意とするプレイヤーを多く生み出しており、エフェクターなど音楽機材の進化と多様化に多大な影響を与えたとも言われている。代表的なギタリストには、ジミー・ペイジ[6]、トニー・アイオミ、エドワード・ヴァン・ヘイレン、マイケル・シェンカー、アンガス・ヤング、イングヴェイ・マルムスティーン、スティーヴ・ヴァイらがいる。通例、強いディストーションをかけ、リフはパワーコードを主体とした力強い音でミュートを効かせながら刻む場合が多い。ヘヴィメタルバンドにはギタリストが2人いることが多い。リズムギター担当とリードギター担当に分かれている場合と、2人が同じリフを弾いて重さを増している場合や、2人が交互にギターソロを弾くこともある。スケールにはペンタトニック、ハーモニック・マ

# **参考サイト**
- [**Pythonによる自然言語処理技法をふんだんに使用した文書要約**](https://software-data-mining.com/python%E3%81%AB%E3%82%88%E3%82%8B%E8%87%AA%E7%84%B6%E8%A8%80%E8%AA%9E%E5%87%A6%E7%90%86%E6%8A%80%E6%B3%95%E3%82%92%E3%81%B5%E3%82%93%E3%81%A0%E3%82%93%E3%81%AB%E4%BD%BF%E7%94%A8%E3%81%97%E3%81%9F/)
- [**sumy で要約(spaCy、GiNZA) を試してみた: Pythonで自然言語処理にフリーライド**](https://itdepends.hateblo.jp/entry/2020/06/07/210222)
- [**ColabでGiNZA v5を試してみた！【固有表現抽出】**](https://dendenblog.xyz/ginza-v5/)
- [**GINZAの公開ページ**](https://megagonlabs.github.io/ginza/)